In [68]:
from contextlib import redirect_stdout

with open('.env_db', 'w') as file:
    with redirect_stdout(file):
        # print(f'DWH_ENDPOINT={DWH_ENDPOINT}')
        # print(f'DWH_ROLE_ARN={DWH_ROLE_ARN}')
        print(a)


In [189]:
from dotenv import load_dotenv
import os
import boto3
import pandas as pd
from pyspark.sql import SparkSession

load_dotenv()


client = boto3.client('s3')
resource = boto3.resource('s3')
bucket='tien-duong1151'
s3_file = 'data/coin_exchange_info.csv'

key = os.getenv('AWS_ACCESS_KEY_ID')
secret = os.getenv('AWS_SECRET_ACCESS_KEY')

# obj = client.get_object(Bucket=bucket, Key=s3_file)
# my_obj = pd.read_csv(obj['Body'])
# my_obj.head(20)
# my_obj.columns = map(lambda x: str(x).capitalize(), my_obj.columns)
# my_obj.rename(columns={'Open':'Open_market'},inplace=True)
# my_obj.replace(to_replace=r'^Panama City Republic of Panama\r\n', value='Panama', regex=True, inplace=True)
# my_obj.replace(to_replace=r'^', value='Panama', regex=True, inplace=True)
# my_obj.drop(columns=['name_id', 'date_added'], inplace=True)
# my_obj = pd.read_csv('data/coins_data.csv')
# my_obj.to_csv(s3_file, index=False)
# # my_obj.head()
def upload_to_s3(bucketname, local_file_path, s3_file_path):

    try:
        s3 = boto3.resource('s3')
        s3.meta.client.upload_file(local_file_path, bucketname, s3_file_path)
        print('Success!')
    except Exception as e:
        print(f'{e}\n Fail!')
upload_to_s3(bucket, s3_file,s3_file)


Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 24
Python-dotenv could not parse statement starting at line 30


Success!


In [66]:
my_obj.query('country.str.contains("Panama")', engine='python')

,id,name,url,country,date_live,usdt,fiat,auto,volume_usd,udate,volume_usd_adj
1,2,1BTCXE,https://1btcxe.com,Panama City Republic of Panama\r\n,2014-01-01,0,1,Yes,0.000000,NaN,0.000000
31,34,Coingi,https://coingi.com/,Panama Bulgaria China US,NaN,0,0,Yes,120.317500,Recently,120.317500
120,149,Idex,https://idex.market,Panama,2017-01-01,0,0,No,15527.603741,Lag,15527.603741


In [72]:
my_obj.replace(to_replace=r'^Panama City Republic of Panama\r\n', value='Panama', regex=True)

,id,name,url,country,date_live,usdt,fiat,auto,volume_usd,udate,volume_usd_adj
0,1,1Broker,https://1broker.com/,Marshall Islands,2011-09-16,0,1,No,0.000000e+00,Recently,0.000000e+00
1,2,1BTCXE,https://1btcxe.com,Panama,2014-01-01,0,1,Yes,0.000000e+00,NaN,0.000000e+00
2,3,ACX,https://acx.io,Australia,2016-04-01,0,1,Yes,0.000000e+00,Recently,0.000000e+00
3,4,ANXPro,https://anxpro.com,Hong Kong China,2014-02-01,0,1,Yes,4.166102e+02,NaN,4.166102e+02
4,5,Binance,https://www.binance.com,Japan,2014-01-01,1,0,Yes,1.417520e+10,Recently,1.481165e+11
...,...,...,...,...,...,...,...,...,...,...,...
285,594,BTCNEXT,http://www.btcnext.io,Saint Vincent and the Grenadines,NaN,1,0,No,2.412004e+06,Lag,2.412004e+06
286,596,Currency.com,https://exchange.currency.com/,Belarus,2019-01-01,1,1,No,3.174756e+08,Recently,1.969422e+07
287,598,Bitci,https://www.bitci.com/,Turkey,2018-01-01,0,0,No,5.134330e+06,Lag,5.134330e+06
288,599,BitLux OTC,https://www.bitlux.exchange/,Russia,NaN,0,1,No,4.297322e+07,Lag,0.000000e+00


In [186]:
import pandas as pd
import requests

def exchange_data_filter(data):
    """[Expect incoming data as Pandas dataframe and return DF as output]
    Args:
        data ([json]): [return exchange data records with country filtered]
    """
    payload = pd.DataFrame.from_dict(data.values())
    payload = payload.astype({'id':int}).copy()
    payload = payload[['id','name', 'url','country','date_live', 'usdt','fiat','auto','volume_usd','udate','volume_usd_adj']].sort_values(by=['id']).set_index('id').query("country != '' ")
    payload['country'].replace(r'[@#&$%+-/\n\r;*]','', regex=True, inplace=True)
    payload.replace({'country':{r'U+K':'United Kingdom ', r'U+S' :'United State ', r'H+K': 'Hong Kong ', r'E+U': 'European Union '}}, regex=True, inplace=True)
    return payload


def get_request(url):
    """[
        - Generate Request from input Url
        ]
    Args:
        url ([type]): [description]
    Returns:
        [type]: [description]
    """
    payload = requests.get(url)
    return payload.json()

        
def export_csv(payload,path):
    """[Used Pandas to export out as CSV format]

    Args:
        payload ([dataframe]): [data to be export]
        path ([None]): [path to be export]
    """
    payload.to_csv(path, index=True)
    print(f'exported to {path}!')


# Exchange data pipeline
coin_exchange_path = 'data/coin_exchange_info.csv'
GET_all_exchanges = 'https://api.coinlore.net/api/exchanges/'
exchange_requests = get_request(GET_all_exchanges)
exchange_data = exchange_data_filter(exchange_requests)
exchange_data
export_csv(exchange_data,coin_exchange_path)


exported to data/coin_exchange_info.csv!


In [188]:
pd.read_csv('data/coin_exchange_info.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              290 non-null    int64  
 1   name            290 non-null    object 
 2   url             290 non-null    object 
 3   country         290 non-null    object 
 4   date_live       220 non-null    object 
 5   usdt            290 non-null    int64  
 6   fiat            290 non-null    int64  
 7   auto            290 non-null    object 
 8   volume_usd      290 non-null    float64
 9   udate           251 non-null    object 
 10  volume_usd_adj  290 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 25.0+ KB


In [163]:
my_list = list(df.replace({'country':{r'U+K':'United Kingdom ', r'U+S' :'United State ', r'H+K': 'Hong Kong ', r'E+U': 'European Union '} }, regex=True).country.unique())
my_list

['Marshall Islands',
 'Panama City Republic of Panama',
 'Australia',
 'Hong Kong China',
 'Japan',
 'Israel',
 'Poland',
 'Indonesia',
 'Hong Kong',
 'South Korea',
 'United Kingdom ',
 'Poland European Union ',
 'Seychelles',
 'Mexico',
 'Luxembourg Luxembourg Slovenia',
 'Seattle Washington United States',
 'Netherlands European Union ',
 'Brazil',
 'Philippines',
 'Turkey',
 'Iceland United State  European Union ',
 'British Virgin Islands China',
 'Thailand',
 'Germany European Union ',
 'London United Kingdom',
 'China',
 'Chile',
 'Tokyo Japan',
 'Panama Bulgaria China United State ',
 'United Kingdom  Czech Republic',
 'India',
 'New Zealand',
 'Spain Russia',
 'San Francisco California United States',
 'United State ',
 'Singapore',
 'Australia New Zealand',
 'United State  United Kingdom  Russia',
 'Ukraine',
 'United Kingdom  Hong Kong',
 'Tanzania',
 'France European Union ',
 'Wilmington Delaware United States',
 'Japan Singapore Vietnam',
 'Argentina',
 'Venezuela',
 'Ita

21/12/28 17:00:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1455190 ms exceeds timeout 120000 ms
21/12/28 17:00:04 WARN SparkContext: Killing executors is not supported by current scheduler.


In [145]:
df.replace({'country':{'UK':'United Kingdom'}})

,name,url,country,date_live,usdt,fiat,auto,volume_usd,udate,volume_usd_adj
id,,,,,,,,,,
1,1Broker,https://1broker.com/,Marshall Islands,2011-09-16,0,1,No,0.000000e+00,Recently,0.000000e+00
2,1BTCXE,https://1btcxe.com,Panama City Republic of Panama,2014-01-01,0,1,Yes,0.000000e+00,NA,0.000000e+00
3,ACX,https://acx.io,Australia,2016-04-01,0,1,Yes,0.000000e+00,Recently,0.000000e+00
4,ANXPro,https://anxpro.com,Hong Kong China,2014-02-01,0,1,Yes,4.166102e+02,NA,4.166102e+02
5,Binance,https://www.binance.com,Japan,2014-01-01,1,0,Yes,1.802082e+10,Recently,1.917998e+11
...,...,...,...,...,...,...,...,...,...,...
594,BTCNEXT,http://www.btcnext.io,Saint Vincent and the Grenadines,None,1,0,No,2.412004e+06,Lag,2.412004e+06
596,Currency.com,https://exchange.currency.com/,Belarus,2019-01-01,1,1,No,8.254338e+08,Recently,5.215666e+07
598,Bitci,https://www.bitci.com/,Turkey,2018-01-01,0,0,No,5.134330e+06,Lag,5.134330e+06


In [2]:
import pandas as pd
df = pd.read_csv('data/coin_market_info.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        974 non-null    object 
 1   base        974 non-null    object 
 2   quote       974 non-null    object 
 3   price       974 non-null    float64
 4   price_usd   974 non-null    float64
 5   volume      974 non-null    float64
 6   volume_usd  974 non-null    float64
 7   time        974 non-null    object 
dtypes: float64(4), object(4)
memory usage: 62.1+ KB


In [140]:
import requests

coin_path = 'data/coins_data.csv'
coin_market_path = 'data/coin_market_info.csv'
coin_exchange_path = 'data/coin_exchange_info.csv'
binance_btc_file = 'data/binance_btc.csv' 

def get_coin_market_request(HowMany=10, DataPath=''):
    """[
        - GET market of top coins
        - Return Pandas DataFrame
        ]   
    Args:
        HowMany (int, optional): [description]. Defaults to 10.
        DataPath (str, optional): [description]. Defaults to ''.
    """
    payload=pd.read_csv(DataPath)
    finalPayload = pd.DataFrame()
    
    df = payload.copy()
    
    df['rank'] = df['rank'].astype(int)
    sort_by_rank = df.sort_values(by='rank')
    top_coins = sort_by_rank.head(HowMany)
    list_of_top_coins = list(top_coins.id)
    
    for i in list_of_top_coins:
        r = requests.get(f'https://api.coinlore.net/api/tickers/?start={i}').json()

        for line in r:
            finalPayload = finalPayload.append(line, ignore_index=True)
    finalPayload['time'] = pd.to_datetime(finalPayload['time'], unit='s')
    
    return finalPayload
       

coin_market_data = get_coin_market_request(HowMany=100, DataPath=coin_path)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [224]:

def get_coin_request(start=0, limit=7000):
    """[
        - Generate requests from API
        - return out as Pandas DataFrame
        ]
    Args:
        start (int, optional): [description]. Defaults to 0.
        limit (int, optional): [description]. Defaults to 100.

    Returns:
        [type]: [description]
    """
    payload = pd.DataFrame()
    for i in range(start,limit,100):
        r = requests.get(f'https://api.coinlore.net/api/tickers/?start={i}').json()
        entry = r['data']
        for line in entry:
            payload = payload.append(line, ignore_index=True)
        print(f"Successfully request number {i}")
    return payload
coin_path='data/coins_data.csv'

coin_requests= get_coin_request(start=0, limit=7000)
export_csv(coin_requests, coin_path)
print("Sucessfully get data and export out to {coin_path}!")

Successfully request number 0
Successfully request number 100
Successfully request number 200
Successfully request number 300
Successfully request number 400
Successfully request number 500
Successfully request number 600
Successfully request number 700
Successfully request number 800
Successfully request number 900
Successfully request number 1000
Successfully request number 1100
Successfully request number 1200
Successfully request number 1300
Successfully request number 1400
Successfully request number 1500
Successfully request number 1600
Successfully request number 1700
Successfully request number 1800
Successfully request number 1900
Successfully request number 2000
Successfully request number 2100
Successfully request number 2200
Successfully request number 2300
Successfully request number 2400
Successfully request number 2500
Successfully request number 2600
Successfully request number 2700
Successfully request number 2800
Successfully request number 2900
Successfully request n

In [229]:
payload=pd.read_csv('data/coins_data.csv')
finalPayload = pd.DataFrame()
df = payload.copy()
df['rank'] = df['rank'].astype(int)
df = df.set_index('id')
df.drop([45577,51539, 48829, 33435], inplace=True)
df.reset_index(inplace=True)
sort_by_rank = df.sort_values(by='rank')
top_coins = sort_by_rank.head()
list_of_top_coins = list(top_coins.id)
df.index.value_counts()

0       1
4661    1
4672    1
4671    1
4670    1
       ..
2327    1
2326    1
2325    1
2324    1
6995    1
Length: 6996, dtype: int64

In [230]:

def top_rank_coins(HowMany=10, DataPath='data/coins_data.csv'):
    """[
        - GET market of top coins
        - Return Pandas DataFrame
        ]   
    Args:
        HowMany (int, optional): [description]. Defaults to 10.
        DataPath (str, optional): [description]. Defaults to ''.
    """
    payload=pd.read_csv(DataPath)
    finalPayload = pd.DataFrame()
    df = payload.copy()
    df['rank'] = df['rank'].astype(int)
    df = df.set_index('id')
    df.drop([45577,51539, 48829, 33435], inplace=True)
    df.reset_index(inplace=True)
    sort_by_rank = df.sort_values(by='rank')
    top_coins = sort_by_rank.head(HowMany)
    list_of_top_coins = list(top_coins.id)
    
    for i in list_of_top_coins:
        r = requests.get(f'https://api.coinlore.net/api/coin/markets/?id={i}').json()
        for line in r:
            finalPayload = finalPayload.append(line, ignore_index=True)
    finalPayload['time'] = pd.to_datetime(finalPayload['time'], unit='s')
    
    return finalPayload
top_coins_path = 'data/top_coins.csv'
coin_market_data = top_rank_coins(HowMany=25)
export_csv(coin_market_data, top_coins_path)
print("Sucessfully get data and export out to: {top_coins_path}!")


exported to data/top_coins.csv!
Sucessfully get data and export out to: {top_coins_path}!


In [184]:
count = 0
for i in list_of_top_coins:
    print(f'https://api.coinlore.net/api/coin/markets/?id={i}\ncount: {count}')
    count+=1
    r = requests.get(f'https://api.coinlore.net/api/coin/markets/?id={i}').json()
    # print(f'https://api.coinlore.net/api/coin/markets/?id={i}')
    for line in r:
        finalPayload = finalPayload.append(line, ignore_index=True)
        
finalPayload['time'] = pd.to_datetime(finalPayload['time'], unit='s')
    

https://api.coinlore.net/api/coin/markets/?id=80
count: 0
https://api.coinlore.net/api/coin/markets/?id=2710
count: 1
https://api.coinlore.net/api/coin/markets/?id=518
count: 2
https://api.coinlore.net/api/coin/markets/?id=48543
count: 3
https://api.coinlore.net/api/coin/markets/?id=257
count: 4
https://api.coinlore.net/api/coin/markets/?id=58
count: 5
https://api.coinlore.net/api/coin/markets/?id=33285
count: 6
https://api.coinlore.net/api/coin/markets/?id=48537
count: 7
https://api.coinlore.net/api/coin/markets/?id=44883
count: 8
https://api.coinlore.net/api/coin/markets/?id=45219
count: 9
https://api.coinlore.net/api/coin/markets/?id=2
count: 10
https://api.coinlore.net/api/coin/markets/?id=45088
count: 11
https://api.coinlore.net/api/coin/markets/?id=48591
count: 12
https://api.coinlore.net/api/coin/markets/?id=33422
count: 13
https://api.coinlore.net/api/coin/markets/?id=47305
count: 14
https://api.coinlore.net/api/coin/markets/?id=1
count: 15
https://api.coinlore.net/api/coin/mar

In [149]:
# https://api.coinlore.net/api/coin/markets/?id=45577
r = requests.get(f'https://api.coinlore.net/api/coin/markets/?id=45577').json()
r

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:
# from pandas_functions import *
# GET_all_exchanges = 'https://api.coinlore.net/api/exchanges/'
# exchange_requests = get_request(GET_all_exchanges)
# payload = pd.DataFrame.from_dict(exchange_requests.values())

In [99]:
coins = pd.read_csv('data/coins_data.csv')
coins.head()

,id,symbol,name,rank,price_usd,percent_change_24h,percent_change_1h,percent_change_7d,price_btc,market_cap_usd,volume24,volume24a,csupply,tsupply,msupply
0,80,ETH,Ethereum,2.0,3975.62,-0.81,0.70,-0.91,0.081236,472398949111.76,1.070368e+10,1.071577e+10,118824085.00,1.188241e+08,NaN
1,2710,BNB,Binance Coin,3.0,532.85,-1.65,0.44,0.22,0.010888,88880203266.66,1.217557e+09,1.124162e+09,166801148.00,1.924433e+08,2.000000e+08
2,518,USDT,Tether,4.0,1.00,0.22,-0.13,0.02,0.000020,73649628616.20,5.086124e+10,4.953762e+10,73578322706.00,6.826127e+10,NaN
3,48543,SOL,Solana,5.0,182.29,-0.82,1.31,2.95,0.003725,55689677513.92,1.302594e+09,1.424015e+09,305506950.00,4.915614e+08,2.100000e+07
4,257,ADA,Cardano,6.0,1.39,3.99,0.84,7.69,0.000028,46245271199.84,9.643069e+08,9.360932e+08,33313246914.00,3.202579e+10,4.500000e+10


In [135]:
list(coins.symbol)

['ETH',
 'BNB',
 'USDT',
 'SOL',
 'ADA',
 'XRP',
 'USDC',
 'LUNA',
 'AVAX',
 'DOT',
 'DOGE',
 'SHIB',
 'BUSD',
 'WBTC',
 'UNI',
 'LTC',
 'LINK',
 'ALGO',
 'BCH',
 'UST',
 'NEAR',
 'XLM',
 'MANA',
 'AXS',
 'ATOM',
 'FTT',
 'VET',
 'TRX',
 'MATIC',
 'CRO',
 'SAND',
 'ETC',
 'FIL',
 'THETA',
 'FTM',
 'XTZ',
 'XMR',
 'GALA',
 'IOTA',
 'EGLD',
 'LEO',
 'GRT',
 'KLAY',
 'EOS',
 'AAVE',
 'HBAR',
 'LRC',
 'STX',
 'CAKE',
 'FLOW',
 'ONE',
 'BTT',
 'MKR',
 'BCHSV',
 'KSM',
 'ENJ',
 'QNT',
 'CRV',
 'ZEC',
 'XEC',
 'NEO',
 'KCS',
 'RUNE',
 'BAT',
 'OKB',
 'AR',
 'CELO',
 'WAVES',
 'KDA',
 'CHZ',
 'CVX',
 'HT',
 'DASH',
 'HOT',
 'COMP',
 'NEXO',
 'LN',
 'TUSD',
 'NEM',
 'YFI',
 'IOTX',
 'OMI',
 'TFUEL',
 'ROSE',
 'DCR',
 'ICX',
 'QTUM',
 'RVN',
 'SAFEMOON',
 'CEL',
 'LPT',
 'OMG',
 'SUSHI',
 'WAX',
 'ZIL',
 'REV',
 'VGX',
 'ELON',
 'RENBTC',
 'AUDIO',
 'SOLO',
 'SC',
 'BNT',
 'RACA',
 'BTG',
 'ILV',
 'ZRX',
 'TEL',
 'STORJ',
 'SNX',
 'GNO',
 'IOST',
 'BOBA',
 'ONT',
 'XWC',
 'UMA',
 'CHSB',
 'RAY',

In [119]:
coins.loc[coins['id'] == ...]

,id,symbol,name,rank,price_usd,percent_change_24h,percent_change_1h,percent_change_7d,price_btc,market_cap_usd,volume24,volume24a,csupply,tsupply,msupply


In [132]:
coins = coins.astype({'id':'str'})
coins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  7000 non-null   object 
 1   symbol              7000 non-null   object 
 2   name                7000 non-null   object 
 3   rank                7000 non-null   float64
 4   price_usd           7000 non-null   float64
 5   percent_change_24h  7000 non-null   float64
 6   percent_change_1h   6984 non-null   float64
 7   percent_change_7d   6994 non-null   float64
 8   price_btc           7000 non-null   float64
 9   market_cap_usd      7000 non-null   object 
 10  volume24            7000 non-null   float64
 11  volume24a           4496 non-null   float64
 12  csupply             7000 non-null   object 
 13  tsupply             5867 non-null   float64
 14  msupply             4129 non-null   float64
dtypes: float64(10), object(5)
memory usage: 820.4+ KB


In [134]:
coins.loc[coins['sym'] == '34416']


,id,symbol,name,rank,price_usd,percent_change_24h,percent_change_1h,percent_change_7d,price_btc,market_cap_usd,volume24,volume24a,csupply,tsupply,msupply
985,34416,PAR,Parachute,987.0,0.002743,-0.81,0.7,-0.91,5.610000e-08,1669688.23,322076.637031,376169.985886,608669936.00,1.000000e+09,NaN


In [80]:
bnb = pd.read_csv('data/Binance_BTCUSDT_minute (1).csv', header=1)
bnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1207621 entries, 0 to 1207620
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   unix         1207621 non-null  int64  
 1   date         1207621 non-null  object 
 2   symbol       1207621 non-null  object 
 3   open         1207621 non-null  float64
 4   high         1207621 non-null  float64
 5   low          1207621 non-null  float64
 6   close        1207621 non-null  float64
 7   Volume BTC   1207621 non-null  float64
 8   Volume USDT  1207621 non-null  float64
 9   tradecount   1207621 non-null  int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 92.1+ MB


In [81]:
eth = pd.read_csv('data/binance_eth.csv', header=1)
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093034 entries, 0 to 1093033
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   unix         1093034 non-null  int64  
 1   date         1093034 non-null  object 
 2   symbol       1093034 non-null  object 
 3   open         1093034 non-null  float64
 4   high         1093034 non-null  float64
 5   low          1093034 non-null  float64
 6   close        1093034 non-null  float64
 7   Volume ETH   1093034 non-null  float64
 8   Volume USDT  1093034 non-null  float64
 9   tradecount   1093034 non-null  int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 83.4+ MB


In [73]:
eth.head()

,unix,date,symbol,open,high,low,close,Volume ETH,Volume USDT,tradecount
0,1640747160000,2021-12-29 03:06:00,ETH/USDT,3821.74,3822.91,3821.74,3822.49,16.0757,61448.795969,68
1,1640747100000,2021-12-29 03:05:00,ETH/USDT,3819.46,3822.65,3819.21,3821.74,143.0225,546434.696611,433
2,1640747040000,2021-12-29 03:04:00,ETH/USDT,3818.56,3819.95,3818.08,3819.46,82.8743,316466.616671,242
3,1640746980000,2021-12-29 03:03:00,ETH/USDT,3817.74,3819.77,3817.51,3818.56,158.9441,606945.635933,370
4,1640746920000,2021-12-29 03:02:00,ETH/USDT,3816.14,3819.49,3816.14,3817.74,202.2607,772260.205406,335


In [82]:
new_df = pd.concat([bnb, eth], ignore_index=True)
new_df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount,Volume ETH
0,1640747160000,2021-12-29 03:06:00,BTC/USDT,48064.16,48076.87,48062.95,48074.99,12.19057,5.859724e+05,131,NaN
1,1640747100000,2021-12-29 03:05:00,BTC/USDT,48022.02,48074.76,48022.01,48064.16,48.85435,2.347259e+06,979,NaN
2,1640747040000,2021-12-29 03:04:00,BTC/USDT,48015.50,48025.75,48014.61,48022.02,9.76903,4.690989e+05,493,NaN
3,1640746980000,2021-12-29 03:03:00,BTC/USDT,48002.02,48015.84,48002.02,48015.50,11.36253,5.455035e+05,530,NaN
4,1640746920000,2021-12-29 03:02:00,BTC/USDT,47972.48,48018.00,47972.47,48002.02,33.00114,1.583952e+06,766,NaN


In [88]:
coin_market_data.time.unique()

array(['2021-09-09T03:56:20.000000000', '2021-09-09T03:55:12.000000000',
       '2021-09-09T03:56:18.000000000', '2021-09-09T03:53:12.000000000',
       '2021-09-09T03:52:17.000000000', '2021-09-09T03:56:22.000000000',
       '2021-09-09T03:54:08.000000000', '2021-09-09T03:54:19.000000000',
       '2021-09-09T03:56:16.000000000', '2021-09-09T03:56:21.000000000',
       '2021-09-09T03:51:21.000000000', '2021-09-09T03:55:10.000000000',
       '2021-09-09T03:44:16.000000000', '2021-09-09T03:54:29.000000000',
       '2021-09-09T03:54:17.000000000', '2021-09-09T03:56:19.000000000',
       '2021-09-09T03:54:13.000000000', '2021-09-09T03:54:10.000000000',
       '2021-09-09T03:56:17.000000000', '2021-09-09T03:50:11.000000000',
       '2021-09-09T03:53:21.000000000', '2021-09-09T03:52:44.000000000',
       '2021-09-09T03:49:04.000000000', '2021-09-09T03:54:22.000000000',
       '2021-09-09T03:54:12.000000000', '2021-09-09T03:49:03.000000000',
       '2021-09-09T03:56:24.000000000', '2021-09-09

In [84]:
new_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2300655 entries, 0 to 2300654
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   unix         int64  
 1   date         object 
 2   symbol       object 
 3   open         float64
 4   high         float64
 5   low          float64
 6   close        float64
 7   Volume BTC   float64
 8   Volume USDT  float64
 9   tradecount   int64  
 10  Volume ETH   float64
dtypes: float64(7), int64(2), object(2)
memory usage: 193.1+ MB


In [199]:
# pd.read_csv('data/Binance_LTCUSDT_minute.csv').to_csv('data/Binance_LTCUSDT_minute.csv', index=False)
# new_df.to_csv('historical_data.csv', header=1)

In [208]:
bnb = pd.read_csv('data/Binance_BNBUSDT_minute.csv')

In [209]:
bnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985017 entries, 0 to 985016
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   unix         985017 non-null  int64  
 1   date         985017 non-null  object 
 2   symbol       985017 non-null  object 
 3   open         985017 non-null  float64
 4   high         985017 non-null  float64
 5   low          985017 non-null  float64
 6   close        985017 non-null  float64
 7   Volume BNB   985017 non-null  float64
 8   Volume USDT  985017 non-null  float64
 9   tradecount   985017 non-null  int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 75.2+ MB


In [204]:
ltc=pd.read_csv('data/Binance_LTCUSDT_minute.csv')
ltc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031090 entries, 0 to 1031089
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   unix         1031090 non-null  int64  
 1   date         1031090 non-null  object 
 2   symbol       1031090 non-null  object 
 3   open         1031090 non-null  float64
 4   high         1031090 non-null  float64
 5   low          1031090 non-null  float64
 6   close        1031090 non-null  float64
 7   Volume LTC   1031090 non-null  float64
 8   Volume USDT  1031090 non-null  float64
 9   tradecount   1031090 non-null  int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 78.7+ MB


In [207]:
bnb.drop(columns=['Unnamed: 0']).to_csv('data/Binance_BNBUSDT_minute.csv', index=False)

In [213]:
dm_1 =  bnb.head()
dm_2 = ltc.head()
dm_3 = eth.head()

new_df = pd.concat([dm_1, dm_2, dm_3], ignore_index=True)

In [214]:
new_df.to_csv('data/demo_data.csv', index=False)